### 2. 용의자/증인 대화 프롬프트 제작

- 용의자: 왜곡된 정보(일부 진실을 변형하거나 은폐)
    - 질문: 제한 없음 or 제약 있음
- 증인: 진실 정보(플레이어에게 힌트가 되는 정보)
    - 질문: 3~5회 제한

In [ ]:
import ollama

In [ ]:
import json
with open('case_file1.json', 'r', encoding='utf-8') as f:
    case = json.load(f)

In [ ]:
# 용의자 정보
def suspect(case, name):
    for suspect in case['용의자']:
        if suspect['개인 정보']['이름'] == name:
            print('용의자 정보: \n', suspect)
            return suspect

In [ ]:
# 용의자와의 대화
def case_file(case, name):
    chat_history = []
    system_instruction = f"""
    당신은 범죄 사건의 용의자입니다. 당신의 정보는 {suspect(case, name)}입니다. 용의자의 입장에서 진술하세요.
    
    ### 지시사항 ###
    - "[5] 진실"에 대한 내용은 AI 내부 참고용이므로 플레이어에게 공개하지 않습니다.
    - 절대로 플레이어 역할로 발화하지 마세요.
    - 플레이어(프로파일러)의 질문에 용의자인 해당 인물의 시점으로만 답해야 합니다.
    - 진술은 일부 진실을 왜곡할 수 있습니다.

    - 대화체를 사용해서 자연스럽게 대답해야 합니다.
    - 대화는 회피하는 느낌을 주세요.
    사건 정보: {case} 
    
    ### 출력 형식 ###

    """
    chat_history.append({"role":"system", "content":system_instruction})
    while True:
        user_input = input('프로파일러: ')
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        chat_history.append({"role":"user", "content":user_input})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=chat_history)

        answer = response['message']['content']
        print('용의자: ', answer)
        chat_history.append({"role":"assistant", "content":answer})

In [ ]:
case_file(case, "잭 스미스")

In [ ]:
# 증인과의 대화
def case_file(case):
    chat_history = []
    system_instruction = f"""
    당신은 아래 범죄 사건의 증인입니다.
    플레이어(프로파일러)의 질문에 해당 인물의 시점으로만 답해야 합니다.
    사실만을 진술해야 하며 플레이어에게 힌트가 되는 정보를 제공해야 합니다.

    사건 정보: {case} 
    
    ### 지시사항 ###
    - "[5] 진실"에 대한 내용은 AI 내부 참고용이므로 플레이어에게 공개하지 않습니다.
    피해자 가족, 이웃 등
    
    ### 출력 형식 ###

    """
    chat_history.append({"role":"system", "content":system_instruction})
    while True:
        user_input = input('프로파일러: ')
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        chat_history.append({"role":"user", "content":user_input})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=chat_history)

        answer = response['message']['content']
        print('증인: ', answer)
        chat_history.append({"role":"assistant", "content":answer})

In [ ]:
if __name__ == '__main__':
    f = open('case_file.json', 'r', encoding='utf-8')
    case = f.read()